In [ ]:
import arcpy
import os
from pathlib import Path
import pandas as pd
from os import path as osPath

arcpy.env.overwriteOutput = True

<h4>Parametrización de Variables</h4>

In [ ]:
ruta_dir = r"C:\docsProyectos\Insumos"
arcpy.env.workspace = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\BD_Consolidada\Consolidacion_RASTER.gdb"
directorio_csv = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\4.Analitica"
nombre_csv = "inventario_ortofotoimagenes.csv"

rutas_ortofotos = []
nombre_ortofotos = []

<h2>Migración de formatos RASTER a BD Geográfica</h2>
<h4>Tratamiento sobre RASTER, formato ECW</h4>

In [ ]:

nombre_archivos_ecw = []

for folder, subfolders, files in os.walk(ruta_dir):
    for file_name in files:

        if file_name.endswith(".ecw"):
            archivo_ecw = os.path.join(folder, file_name)
            ecw = Path(archivo_ecw)
            nombre_archivos_ecw.append(ecw.stem)
            # ** Para Metadata
            rutas_ortofotos.append(archivo_ecw)
            nombre_ortofotos.append(ecw.stem)
            
        if file_name.endswith(".ecw") or file_name.endswith(".eww") or file_name.endswith(".prj"):
            otros_archivos = os.path.join(folder, file_name)
            p = Path(otros_archivos)

            for i in nombre_archivos_ecw:
                if i == p.stem:
                    # ! Si existe una parametrizacion por presencia de tíldes o espacios, se hace aquí
                    pass        

            if file_name.endswith(".ecw"):
                archivo_ecw = os.path.join(folder, file_name)
                ruta_ecw = Path(archivo_ecw)

                if arcpy.Exists(os.path.join(arcpy.env.workspace, ruta_ecw.stem)):                
                    print("La imagen {0} ya se encuentra centralizada".format(ruta_ecw.name))
                else:
                    arcpy.conversion.RasterToGeodatabase(archivo_ecw, arcpy.env.workspace)
                    print("Se migra {0}".format(ruta_ecw.name))
                    inDataset = os.path.join(arcpy.env.workspace, ruta_ecw.stem)
                    extension = ".prj"
                    ruta_ecw_w_prj = ruta_ecw.stem + extension
                    coorSystem = os.path.join(folder, ruta_ecw_w_prj)
                    coordP = Path(coorSystem)
                    arcpy.management.DefineProjection(in_dataset = inDataset, coor_system = coorSystem)
                    print("Se parametriza .prj {0}".format(coordP.name))

<h4>Tratamiento sobre RASTER, formato TIF</h4>

In [ ]:
nombre_archivos_tif = []

for folder, subfolders, files in os.walk(ruta_dir):
    for file_name in files:

        if file_name.endswith(".tif"):
            archivos_tif = os.path.join(folder, file_name)
            tif = Path(archivos_tif)
            
            nombre_tif = tif.stem
            extension_tif = tif.suffix
            nTifEstandarizado = nombre_tif.replace(',','').replace('.','').replace('-','_').replace(' ','_').replace('á','a').replace('ó','o')
            ruta_nTifEstandarizado = os.path.join(folder, nTifEstandarizado + extension_tif)
            tif_estandarizado = Path(ruta_nTifEstandarizado)
            os.rename(archivos_tif, ruta_nTifEstandarizado)
            print("Se estandariza {0} a {1}".format(tif.name, tif_estandarizado.name))

        if file_name.endswith(".tif"):
            archivos_tif = os.path.join(folder, file_name)
            tif = Path(archivos_tif)
            # ** Para Metadata
            rutas_ortofotos.append(archivos_tif)
            nombre_ortofotos.append(tif.stem)

            if arcpy.Exists(os.path.join(arcpy.env.workspace, tif.stem)):                
                print("La imagen {0} ya se encuentra centralizada".format(tif.name))
            else:
                arcpy.conversion.RasterToGeodatabase(archivos_tif, arcpy.env.workspace)
                print("Se migra {0}".format(tif.name))

<h4>Creación de reporte con información migrada a BD Geográfica</h4>

In [ ]:
lista_df_ortofotos = list(zip(rutas_ortofotos, nombre_ortofotos))
df_ortofotos = pd.DataFrame(lista_df_ortofotos, columns=['ruta_ortofoto','nombre_ortofoto'])
df_ortofotos.to_csv(os.path.join(directorio_csv, nombre_csv))
print("Se genera CSV en directorio de Analítica")